## Storm Data


Recently, the International Hurricane Watchgroup (IHW) has been asked to update their analysis tools. Because of the increase in public awareness of hurricanes, they are required to be more diligient with the analysis of historical hurricane data they share across the organization. They have asked you, someone with experience in databases, to help work with their team to productionize their services.



Data Dictionary:

    fid - ID for the row
    year - Recorded year
    month - Recorded month
    day - Recorded date
    ad_time - Recorded time in UTC
    btid - Hurricane ID
    name - Name of the hurricane
    lat - Latitude of the recorded location
    long - Longitude of the recorded location
    wind_kts - Wind speed in knots per second
    pressure - Atmospheric pressure of the hurricane
    cat - Hurricane category
    basin - The basin the hurricane is located
    shape_leng - Hurricane shape length

### AIM

** 1) Records clenaing.<br>
** 2) Clean records store into csv files.<br>
** 3) Connection with PostgreSQL. <br>
** 4) Create table in PostgreSQL. <br>
** 5) Insert records from clean CSV files.<br>
** 6) Read records.<br>
   7) Update records.<br>

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("DataSet/storm_data.csv")
df.head()

,FID,YEAR,MONTH,DAY,AD_TIME,BTID,NAME,LAT,LONG,WIND_KTS,PRESSURE,CAT,BASIN,Shape_Leng
0,2001,1957,8,8,1800Z,63,NOTNAMED,22.5,-140.0,50,0,TS,Eastern Pacific,1.140175
1,2002,1961,10,3,1200Z,116,PAULINE,22.1,-140.2,45,0,TS,Eastern Pacific,1.166190
2,2003,1962,8,29,0600Z,124,C,18.0,-140.0,45,0,TS,Eastern Pacific,2.102380
3,2004,1967,7,14,0600Z,168,DENISE,16.6,-139.5,45,0,TS,Eastern Pacific,2.121320
4,2005,1972,8,16,1200Z,251,DIANA,18.5,-139.8,70,0,H1,Eastern Pacific,1.702939


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59228 entries, 0 to 59227
Data columns (total 14 columns):
FID           59228 non-null int64
YEAR          59228 non-null int64
MONTH         59228 non-null int64
DAY           59228 non-null int64
AD_TIME       59228 non-null object
BTID          59228 non-null int64
NAME          59228 non-null object
LAT           59228 non-null float64
LONG          59228 non-null float64
WIND_KTS      59228 non-null int64
PRESSURE      59228 non-null int64
CAT           59228 non-null object
BASIN         59228 non-null object
Shape_Leng    59228 non-null float64
dtypes: float64(3), int64(7), object(4)
memory usage: 6.3+ MB


In [5]:
df.describe()

,FID,YEAR,MONTH,DAY,BTID,LAT,LONG,WIND_KTS,PRESSURE,Shape_Leng
count,59228.000000,59228.000000,59228.000000,59228.000000,59228.000000,59228.000000,59228.000000,59228.000000,59228.000000,59228.000000
mean,29614.500000,1957.194874,8.540521,15.867326,648.398899,23.526400,-83.196863,54.726802,372.336800,1.201987
std,17097.795209,41.665792,1.364174,8.793432,372.376803,9.464955,37.282152,25.133577,480.562974,0.834497
min,1.000000,1851.000000,1.000000,1.000000,1.000000,4.200000,-180.000000,10.000000,0.000000,0.000000
25%,14807.750000,1928.000000,8.000000,8.000000,344.000000,16.100000,-108.500000,35.000000,0.000000,0.707107
50%,29614.500000,1970.000000,9.000000,16.000000,606.000000,21.200000,-81.200000,50.000000,0.000000,1.029563
75%,44421.250000,1991.000000,9.000000,23.000000,920.000000,29.600000,-62.200000,70.000000,990.000000,1.431782
max,59228.000000,2008.000000,12.000000,31.000000,1410.000000,69.000000,180.000000,165.000000,1024.000000,11.180340


##### We can infer most of these values through inspection of the min and max values in df.describe(). We could then produce a table 

In [29]:
df['CAT'].str.len().max()

2

In [28]:
df['BASIN'].str.len().max()

15

This means that these columns are maximum 15 and 2 characters long. Out table will end looking like

#### Clean missing values

In [30]:
# total na numbers
df.isna().sum()

FID           0
YEAR          0
MONTH         0
DAY           0
AD_TIME       0
BTID          0
NAME          0
LAT           0
LONG          0
WIND_KTS      0
PRESSURE      0
CAT           0
BASIN         0
Shape_Leng    0
dtype: int64

In [31]:
df.isnull().sum()

FID           0
YEAR          0
MONTH         0
DAY           0
AD_TIME       0
BTID          0
NAME          0
LAT           0
LONG          0
WIND_KTS      0
PRESSURE      0
CAT           0
BASIN         0
Shape_Leng    0
dtype: int64

AD_TIME from the current format to a valid datetime

In [32]:
df['AD_TIME'].head()

0    1800Z
1    1200Z
2    0600Z
3    0600Z
4    1200Z
Name: AD_TIME, dtype: object

In [34]:
df['TIMESTAMP'] = df['YEAR'].astype(str) + \
                  df['MONTH'].astype(str).str.zfill(2) + \
                  df['DAY'].astype(str).str.zfill(2) + \
                  df['AD_TIME']
df['TIMESTAMP'].head()

0    195708081800Z
1    196110031200Z
2    196208290600Z
3    196707140600Z
4    197208161200Z
Name: TIMESTAMP, dtype: object

In [35]:
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'], format='%Y%m%d%H%MZ', utc=True)

In [36]:
df['TIMESTAMP'].head()

0   1957-08-08 18:00:00+00:00
1   1961-10-03 12:00:00+00:00
2   1962-08-29 06:00:00+00:00
3   1967-07-14 06:00:00+00:00
4   1972-08-16 12:00:00+00:00
Name: TIMESTAMP, dtype: datetime64[ns, UTC]

In [37]:
df = df.drop(['YEAR','MONTH','DAY','AD_TIME'], axis=1)
df.head()

,FID,BTID,NAME,LAT,LONG,WIND_KTS,PRESSURE,CAT,BASIN,Shape_Leng,TIMESTAMP
0,2001,63,NOTNAMED,22.5,-140.0,50,0,TS,Eastern Pacific,1.140175,1957-08-08 18:00:00+00:00
1,2002,116,PAULINE,22.1,-140.2,45,0,TS,Eastern Pacific,1.166190,1961-10-03 12:00:00+00:00
2,2003,124,C,18.0,-140.0,45,0,TS,Eastern Pacific,2.102380,1962-08-29 06:00:00+00:00
3,2004,168,DENISE,16.6,-139.5,45,0,TS,Eastern Pacific,2.121320,1967-07-14 06:00:00+00:00
4,2005,251,DIANA,18.5,-139.8,70,0,H1,Eastern Pacific,1.702939,1972-08-16 12:00:00+00:00


In [41]:
df['BASIN'].astype('category').cat.codes.unique()
df['CAT'].astype('category').cat.codes.unique()

array([10,  1,  9,  3,  2,  6,  4,  5,  0, 11,  8,  7], dtype=int64)

In [44]:
df.columns = df.columns.str.upper()

In [45]:
# we rearrange our columns and reset the dataframe's index
df_csv = df[['FID', 'TIMESTAMP', 'BTID', 'NAME', 'LAT', 'LONG', 'WIND_KTS', 'PRESSURE', 'CAT', 'BASIN', 'SHAPE_LENG']].copy()
df_csv = df_csv.set_index('FID', drop=True)
df_csv.head()

,TIMESTAMP,BTID,NAME,LAT,LONG,WIND_KTS,PRESSURE,CAT,BASIN,SHAPE_LENG
FID,,,,,,,,,,
2001,1957-08-08 18:00:00+00:00,63,NOTNAMED,22.5,-140.0,50,0,TS,Eastern Pacific,1.140175
2002,1961-10-03 12:00:00+00:00,116,PAULINE,22.1,-140.2,45,0,TS,Eastern Pacific,1.166190
2003,1962-08-29 06:00:00+00:00,124,C,18.0,-140.0,45,0,TS,Eastern Pacific,2.102380
2004,1967-07-14 06:00:00+00:00,168,DENISE,16.6,-139.5,45,0,TS,Eastern Pacific,2.121320
2005,1972-08-16 12:00:00+00:00,251,DIANA,18.5,-139.8,70,0,H1,Eastern Pacific,1.702939


In [48]:
df_csv.to_csv(path_or_buf='DataSet/cleaned_storm_data.csv', sep=';')

## Creating the table with SQL

In [26]:
import psycopg2
connection = psycopg2.connect(user = "postgres",
                                  password = "12345",
                                  host = "localhost",
                                  database = "med_rpr_stg")
cursor = connection.cursor()
# Print PostgreSQL Connection properties
print ( connection.get_dsn_parameters(),"\n")

{'user': 'postgres', 'dbname': 'med_rpr_stg', 'host': 'localhost', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [38]:
# Print PostgreSQL version


CREATE_TABLE_SQL ="""
CREATE TABLE hurricanes (
    fid INT PRIMARY KEY,
    timestamp TIMESTAMP WITH TIME ZONE,
    btid SMALLINT,
    name VARCHAR,
    lat DECIMAL(8,6),
    long DECIMAL(9,6),
    wind_kts SMALLINT,
    pressure SMALLINT,
    cat VARCHAR(2),
    basin VARCHAR(16),
    shape_length DECIMAL(8, 6)
    )
"""


select_Business_1 = cursor.execute(CREATE_TABLE_SQL)

connection.commit()

### Insert the data in the database

In [51]:
with open('DataSet/cleaned_storm_data.csv', 'r+') as f:
    cursor.copy_expert("COPY hurricanes FROM STDIN WITH CSV HEADER DELIMITER ';'", f)


In [52]:
cursor.execute('SELECT * FROM hurricanes LIMIT 1')
cursor.fetchall()

[(2001,
  datetime.datetime(1957, 8, 8, 23, 30, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=330, name=None)),
  63,
  'NOTNAMED',
  Decimal('22.500000'),
  Decimal('-140.000000'),
  50,
  0,
  'TS',
  'Eastern Pacific',
  Decimal('1.140175'))]

### Should be able to insert data

In [53]:
cursor.execute('INSERT INTO hurricanes (fid) values (1000001)')
connection.commit()

### Read this new row from the table

In [54]:
# we can
cursor.execute('SELECT * FROM hurricanes WHERE fid = 1000001')
cursor.fetchall()

[(1000001, None, None, None, None, None, None, None, None, None, None)]

In [55]:
cursor.execute("UPDATE hurricanes SET name = 'fakename' WHERE fid=1000001")
connection.commit()

### Read the updated changes back

In [56]:
cursor.execute('SELECT * FROM hurricanes WHERE fid = 1000001')
cursor.fetchall()

[(1000001, None, None, 'fakename', None, None, None, None, None, None, None)]